In [5]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/root/autodl-tmp/FreeDARTS-main/NAS_Bench201/lib'))
sys.path.append(str('/root/autodl-tmp/FreeDARTS-main/NAS_Bench201/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API
import torch.nn.functional as F
from weight_initializers import init_net


def pruning_func(xloader, network, criterion, scheduler, w_optimizer, a_optimizer, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()   
      

    for name, param in network.module.state_dict().items():
        if name=='arch_parameters':###whether abs on arch parameter
            print(param)
            param.abs_()
        else:
            param.abs_()
    arch_pruned=deepcopy(network.module.arch_parameters.data)
    
    
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time

        # update the architecture-weight
        a_optimizer.zero_grad()
        
        input_dim = list(arch_inputs[0,:].shape)
        inputs = torch.ones([1] + input_dim).float().cuda(non_blocking=True)
        _, logits = network.forward(inputs)        
        
        arch_loss = torch.sum(logits)
        arch_loss.backward()
        #a_optimizer.step()
        #arch_synflow=abs(F.softmax(network.module.arch_parameters.data)*network.module.arch_parameters.grad.data)
        #arch_synflow=abs(network.module.arch_parameters.grad.data)        
        arch_synflow=abs(network.module.arch_parameters.data*network.module.arch_parameters.grad.data)        
        print(network.module.arch_parameters.data)
        print(network.module.arch_parameters.grad.data)   
        
        break
          
            
    return arch_synflow


parser = argparse.ArgumentParser("DARTS First Order")
parser.add_argument('--data_path',          type=str,   default= '/root/autodl-tmp', help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10', choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102',help='The search space name.')
parser.add_argument('--max_nodes',          type=int,   default= 4 ,help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16,help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
# architecture leraning rate
parser.add_argument('--arch_learning_rate', type=float, default=3e-2, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay',  type=float, default=1e-3, help='weight decay for arch encoding')
# log
parser.add_argument('--workers',            type=int,   default=2,    help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/root/autodl-tmp/NAS-Bench-201-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=50,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=242, help='manual seed')
args = parser.parse_args([])

In [6]:
args.save_dir= './00FreeDARTS_SynFlow_seed_'+str(args.rand_seed)+args.save_dir

xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.cuda.set_device(0)
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/root/autodl-tmp/FreeDARTS-main/NAS_Bench201/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
elif xargs.dataset.startswith('ImageNet16'):
    split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
    imagenet16_split = load_config(split_Fpath, None, None)
    train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
config_path = '/root/autodl-tmp/FreeDARTS-main/NAS_Bench201/configs/nas-benchmark/algos/DARTS.config'
config = load_config(config_path, {'class_num': class_num, 'xshape': xshape}, logger)
# To split data
train_data_v2 = deepcopy(train_data)
train_data_v2.transform = valid_data.transform
valid_data    = train_data_v2
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
valid_loader  = torch.utils.data.DataLoader(valid_data, batch_size=config.batch_size, sampler=torch.utils.data.sampler.SubsetRandomSampler(valid_split), num_workers=xargs.workers, pin_memory=True)
logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'DARTS-V1', 'C': xargs.channel, 'N': xargs.num_cells,
                              'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                              'space'    : search_space}, None)
search_model = get_cell_based_tiny_net(model_config)
logger.log('search-model :\n{:}'.format(search_model))


w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.get_weights(), config)
a_optimizer = torch.optim.Adam(search_model.get_alphas(), lr=xargs.arch_learning_rate, betas=(0.5, 0.999), weight_decay=xargs.arch_weight_decay)


logger.log('w-optimizer : {:}'.format(w_optimizer))
logger.log('a-optimizer : {:}'.format(a_optimizer))
logger.log('w-scheduler : {:}'.format(w_scheduler))
logger.log('criterion   : {:}'.format(criterion))
flop, param  = get_model_infos(search_model, xshape)
#logger.log('{:}'.format(search_model))
logger.log('FLOP = {:.2f} M, Params = {:.2f} MB'.format(flop, param))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()


  # start training
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
epoch=0
w_scheduler.update(0, 0.0)

arch_pruned = pruning_func(search_loader, network, criterion, w_scheduler, w_optimizer, a_optimizer, xargs.print_freq, logger)
search_time.update(time.time() - start_time)
logger.log('Pruning cost {:.1f} s.'.format(search_time.sum))
logger.log('Pruning finished')
logger.log('-----------------------------------------------------------------')
if api is not None: logger.log('{:}'.format(api.query_by_arch(network.module.genotype_prune(arch_pruned)))) 
logger.close()

Main Function with logger : Logger(dir=00FreeDARTS_SynFlow_seed_242/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
arch_learning_rate : 0.03
arch_nas_dataset : /root/autodl-tmp/NAS-Bench-201-v1_0-e61699.pth
arch_weight_decay : 0.001
channel          : 16
data_path        : /root/autodl-tmp
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 50
rand_seed        : 242
save_dir         : ./00FreeDARTS_SynFlow_seed_242/output/search-cell-nas-bench-102-cifar10
search_space_name : nas-bench-102
workers          : 2
Python  Version  : 3.8.10 (default, Jun  4 2021, 15:09:15)  [GCC 7.5.0]
Pillow  Version  : 9.1.1
PyTorch Version  : 1.11.0+cu113
cuDNN   Version  : 8200
CUDA available   : True
CUDA GPU numbers : 1
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloaded and verified
Load split file from /root/autodl-tmp/FreeDARTS-main/NAS_Bench201/configs/nas-ben

In [7]:
network.module.arch_parameters.data*network.module.arch_parameters.grad.data

tensor([[ 0.0000e+00, -3.9850e-04, -7.1574e-02,  2.2406e-01,  1.0441e-02],
        [ 0.0000e+00,  3.0956e-03, -2.6816e-01,  1.0208e-01,  8.1223e-04],
        [ 0.0000e+00,  3.8474e-06, -2.2047e-02,  2.3045e-01,  7.3678e-06],
        [ 0.0000e+00,  5.7351e-03,  2.1198e-01,  1.2254e-01,  7.5851e-03],
        [ 0.0000e+00,  1.7748e-04,  1.9114e-01,  1.5770e-01,  3.2269e-05],
        [ 0.0000e+00,  7.2769e-04,  3.9169e-01,  9.5154e-01,  3.1059e-03]],
       device='cuda:0')

In [ ]:
network.module.arch_parameters.grad.data

In [ ]:
network.module.arch_parameters.data